In [7]:
from tfim_sk_infd.services import jij_service
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import numpy
import sys

numpy.set_printoptions(threshold=sys.maxsize)

T = 4

Jij_instance = jij_service.get_toy_Jij(T)
SG = SKSpinGlass(Jij_instance)

print(SG.JZZ.get_shape())

print(SG.JZZ.diagonal() )

100% |########################################################################|


Building JZZ...
Done.
Building Mx...
Done.
(16, 16)
[-2.  2.  2.  2.  0.  0.  0. -4. -4.  0.  0.  0.  2.  2.  2. -2.]


In [4]:
import numpy as np

# Original NumPy array
original_array = np.array([[1, 2, 3], [4, 5, 6]])

# Convert the NumPy array to a string
array_string = np.array2string(original_array)

In [5]:
print(array_string)

[[1 2 3]
 [4 5 6]]


In [9]:
import json
original_array = np.array([[1, 2, 3], [4, 5, 6]])

arr_str = json.dumps(original_array.tolist())
np.array(json.loads(arr_str))

array([[1, 2, 3],
       [4, 5, 6]])

In [10]:
N=8
(N*(N-1)) / 2

28.0

In [47]:
from tfim_sk_infd.services import jij_service, ground_state_service, qutip_service
import numpy as np

# Get an instance and find ground state
seed = 0
N = 8
Jij = jij_service.Jij_instance(N, seed)
JZZ = jij_service.JZZ_SK(Jij)
gs_list = np.where(JZZ.diagonal() == JZZ.diagonal().max())[0]

# Pull out half of the ground states and determine maximum hd
reduced_gs, max_h_d = ground_state_service.maximal_half_clique(gs_list, N)

print(reduced_gs)

gs_size = len(reduced_gs)

# Find quantum annealing final state
Hp = qutip_service.spin_glass_hamiltonian(Jij, N)
Ht = qutip_service.transverse_field_hamiltonian(N)

h = 1e-2
h_t = (-Hp*(1-h)) - (h * Ht)

eigenvalues, eigenstates = h_t.eigenstates(eigvals = 1)

# print(eigenstates[0])

annealing_gs_probs = qutip_service.quantum_anneal(Hp, Ht, N, 1000, 100)

[9, 30]


In [48]:
eigenstates[0].full().flatten()[gs_list]

array([0.49998355+0.j, 0.499984  +0.j, 0.49998385+0.j, 0.49998421+0.j])

In [34]:
np.array(annealing_gs_probs)[gs_list]

array([0.24999996, 0.24999996, 0.24999996, 0.24999996])

In [51]:
for h in (1, 0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 ):
    h_t = (-Hp*(1-h)) - (h * Ht)

    # Calculate condition number
    cond_number = np.linalg.cond(h_t.full())
    print("Condition number:", cond_number)

print(np.linalg.cond(Hp.full()))

Condition number: 8.621715046583702e+17
Condition number: inf
Condition number: 1.8073537748923588e+16
Condition number: 1.1143847909944384e+17
Condition number: 6.501768394827755e+16
Condition number: 1.2890019807250006e+17
Condition number: 1.2631674856366051e+17
inf


In [60]:
h_test = Hp.full()[1:6,1:6]
print(h_test)
print(np.linalg.cond(h_test))

[[ 6.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -6.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -4.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  4.+0.j]]
3.0
